In [8]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install --upgrade protobuf


   ---------------------------------------- 0.0/426.9 kB ? eta -:--:--
   -- ------------------------------------- 30.7/426.9 kB 1.3 MB/s eta 0:00:01
   ------ -------------------------------- 71.7/426.9 kB 787.7 kB/s eta 0:00:01
   ------------- -------------------------- 143.4/426.9 kB 1.1 MB/s eta 0:00:01
   -------------- ------------------------- 153.6/426.9 kB 1.0 MB/s eta 0:00:01
   --------------------- ------------------ 225.3/426.9 kB 1.1 MB/s eta 0:00:01
   ------------------------- -------------- 276.5/426.9 kB 1.0 MB/s eta 0:00:01
   --------------------------- ---------- 307.2/426.9 kB 999.9 kB/s eta 0:00:01
   ---------------------------- --------- 317.4/426.9 kB 936.6 kB/s eta 0:00:01
   ------------------------------- ------ 358.4/426.9 kB 890.4 kB/s eta 0:00:01
   ------------------------------- ------ 358.4/426.9 kB 890.4 kB/s eta 0:00:01
   -------------------------------------- 426.9/426.9 kB 859.1 kB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existin

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.14 requires protobuf<5,>=4.25.3, but you have protobuf 5.27.3 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.27.3 which is incompatible.
tensorflow-intel 2.16.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.27.3 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
%pip uninstall mediapipe -y
%pip install mediapipe


Found existing installation: mediapipe 0.10.14
Uninstalling mediapipe-0.10.14:
  Successfully uninstalled mediapipe-0.10.14
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/50.8 MB 330.3 kB/s eta 0:02:34
   ---------------------------------------- 0.1/50.8 MB 656.4 kB/s eta 0:01:18
   ---------------------------------------- 0.1/50.8 MB 819.2 kB/s eta 0:01:02
   ---------------------------------------- 0.1/50.8 MB 718.6 kB/s eta 0:01:11
   ---------------------------------------- 0.2/50.8 MB 857.5 kB/s eta 0:01:00
   ---------------------------------------- 0.2/50.8 MB 772.8 kB/s eta 0:01:06
   ---------------------------------------- 0.3/50.8 MB 853.3 kB/s eta 0:01:00
   ---------------------------------------- 0.3/50.8 MB 936.6 kB/s eta 0:00:54
   ---------------------------------------- 0.4/50.8 MB 983.6 kB/s eta 0:00:52
   ---------------------------------------- 0.4/50.8 MB 1.0 MB/s eta 0:00:50
   ---------------------------------------- 0.5/50.8 MB 994.4 kB/s eta 0:00:51
   ---------------------------------------- 0.6/50.8 MB 1.1 MB/


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip cache purge
%pip install --no-cache-dir mediapipe protobuf


Files removed: 37
Note: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:


import cv2
import mediapipe as mp
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [13]:
# Initialize the webcam
cap = cv2.VideoCapture(0)

# Capture a few images for storing landmarks
captured_images = []

for i in range(5):
    ret, frame = cap.read()
    if ret:
        captured_images.append(frame)
        cv2.imshow('Captured Image', frame)
        cv2.waitKey(1000)  # Wait for 1 second between captures

cap.release()
cv2.destroyAllWindows()


In [15]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

def extract_landmarks(image):
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = []
            for landmark in face_landmarks.landmark:
                landmarks.append((landmark.x, landmark.y, landmark.z))
            return np.array(landmarks).flatten()
    return None

stored_landmarks = [extract_landmarks(img) for img in captured_images]


In [16]:
# Save the stored landmarks for future use
np.save('stored_landmarks.npy', np.array(stored_landmarks))


In [17]:
# Load stored landmarks
stored_landmarks = np.load('stored_landmarks.npy')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    live_landmarks = extract_landmarks(frame)

    if live_landmarks is not None:
        similarities = [cosine_similarity([live_landmarks], [stored]) for stored in stored_landmarks]
        average_similarity = np.mean(similarities)

        cv2.putText(frame, f'Similarity: {average_similarity:.2f}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        if average_similarity > 0.8:  # Threshold for "authentication"
            cv2.putText(frame, 'Authenticated', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, 'Not Authenticated', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\ASUS\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\ASUS\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\ASUS\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\ASUS\anaconda3\Lib\site-packages\google\pro